In [1]:
# pytorch - 4

# ANN using NN Module (nn.Module)

# nn.Linear     : fully connected / Dense layer
# nn.Relu       : Hidden layer activation fn
# nn.Sigmoid    : output layer activation fn
# nn.Sequential : container
# nn.BCELoss    : Binary CrossEntropy Loss fn
# torch.optim   : optimizer

In [2]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,LabelEncoder

In [3]:
# data loading
# breast cancer

url = 'https://raw.githubusercontent.com/gscdit/Breast-Cancer-Detection/master/data.csv'
df = pd.read_csv(url)

df.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


In [4]:
# dropping unwanted columns

df.drop(['id','Unnamed: 32'],axis=1,inplace=True)

In [5]:
# defining features & target

X = df.drop('diagnosis',axis=1)
y = df['diagnosis']

In [6]:
# encoding target

le = LabelEncoder()
y = le.fit_transform(y)

# B : Non-Cancerous : 0
# M : Cancerous     : 1

In [7]:
# train-test split

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=38)

In [8]:
# scaling

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [9]:
# to tensors

X_train_t = torch.from_numpy(X_train).float()
X_test_t = torch.from_numpy(X_test).float()

y_train_t = torch.from_numpy(y_train).float().unsqueeze(1)
y_test_t = torch.from_numpy(y_test).float().unsqueeze(1)

In [10]:
print(X_train_t.shape)
print(X_test_t.shape)

print(y_train_t.shape)
print(y_test_t.shape)

torch.Size([455, 30])
torch.Size([114, 30])
torch.Size([455, 1])
torch.Size([114, 1])


In [11]:
# defining simple NN

class SimpleNN(nn.Module):

  def __init__(self,n_feats):          # constructor
    super().__init__()                 # invoke nn.Module constructor

    self.network = nn.Sequential(
        nn.Linear(n_feats,3),          # 30 feats ; 3 nuerons = 3 output
        nn.ReLU(),
        nn.Linear(3,1),                # above 3 output as 3 input ; 1 neuron
        nn.Sigmoid()
    )

  def forward(self,x):
    out = self.network(x)
    return out                         # out = y_pred

In [12]:
learning_rate = 0.1

epochs = 25

loss_fn = nn.BCELoss()            # loss function

In [13]:
# training pipeline

model = SimpleNN(X_train_t.shape[1])

optimizer = optim.SGD(model.parameters(), lr=learning_rate)          # optimizer

for epoch in range(epochs):

  y_pred = model.forward(X_train_t)                               # forward pass

  loss = loss_fn(y_pred,y_train_t)                            # loss calculation

  optimizer.zero_grad()                                         # zero gradients

  loss.backward()                                             # back propagation

  optimizer.step()                                              # params updates

  print(f"Epoch: {epoch + 1} Loss: {loss.item()}")

Epoch: 1 Loss: 0.7865103483200073
Epoch: 2 Loss: 0.7677655816078186
Epoch: 3 Loss: 0.7507419586181641
Epoch: 4 Loss: 0.7347407937049866
Epoch: 5 Loss: 0.7194616794586182
Epoch: 6 Loss: 0.7043989300727844
Epoch: 7 Loss: 0.6894787549972534
Epoch: 8 Loss: 0.6744966506958008
Epoch: 9 Loss: 0.6592612266540527
Epoch: 10 Loss: 0.6434131264686584
Epoch: 11 Loss: 0.6270854473114014
Epoch: 12 Loss: 0.6100841760635376
Epoch: 13 Loss: 0.5924896001815796
Epoch: 14 Loss: 0.5745139122009277
Epoch: 15 Loss: 0.5560689568519592
Epoch: 16 Loss: 0.5373210310935974
Epoch: 17 Loss: 0.5184326767921448
Epoch: 18 Loss: 0.4996315538883209
Epoch: 19 Loss: 0.4809511601924896
Epoch: 20 Loss: 0.462600439786911
Epoch: 21 Loss: 0.44467464089393616
Epoch: 22 Loss: 0.42737701535224915
Epoch: 23 Loss: 0.41077953577041626
Epoch: 24 Loss: 0.39488303661346436
Epoch: 25 Loss: 0.37967658042907715


In [14]:
# evaluation

model.eval()                                                              # switch to evaluation mode

with torch.no_grad():

    y_pred = model(X_test_t)

    y_pred_labels = (y_pred > 0.5).float()

    accuracy = (y_pred_labels == y_test_t).float().mean()

    print(f'Test Accuracy: {accuracy.item() * 100:.2f}%')


Test Accuracy: 93.86%


model.eval()

By default,model.train() : "Training Mode." <br>
model.eval() tells PyTorch to change the behavior of specific layers:

BatchNorm: It stops calculating the mean/variance of the current batch and starts using the "running averages" it learned during training.

Dropout: stops dropout